# CLI

```
poodle [Files/Folders to mutate]
    --runner {runner name or path}
    --report {report name, repeatable}  # replaces default list of reporters
    --max_parallel -P {number max parallel runners}
    --base_folder_name {}
```

# Config 

## poodle_config.py
```
targets: list[str] = [File and Folder names to mutate]

runner = Runner Name str or Runner Object

# all mutators enabled by default ???
register_mutators: dict = {
    Name: object
} 
disable_mutators: list[str] = [Names to disable]
enable_mutators: list[str] = [Names to enable]

# only enable sysout by default ???
register_reporters = {
    Name: object
}
disable_reporters: list[str] = [Names to disable]???
enable_reporters: list[str] = [Names to enable]

max_parallel: int = default to system.affil cup?

base_folder: str = "poodle-runner"
```

# files
cli.py
* Responsible for CLI interface and calling core.
core.py
* Resolve configuration (defaults, poodle_config.py, toml, args)
* Init Mutators
* Find list of ".py" files to mutate
* Parse Py file, pass to each mutator, build list of PoodleMutant objects
* Create pool of Runners
* As each runner is available, call with PoodleMutant, aggregate results
* Print out current status every few seconds
* When all done, Init and call Reporter

# Mutator
* If class, will init with kwargs: poodle_config: dict
* create_mutants(module: ast.Module, poodle_config: dict) -> list or generator? [ast.Module or PoodleMutant?]

# Runner
* Must be class, will init with kwargs: folder: Path, poodle_config: dict
* test_mutation(mutant: PoodleMutant, timeout_s: int) -> (pass: bool, timeout: bool, reason: str | None)

# Reporter
* If class, will init with kwargs: poodle_config: dict
* create_report(report: PoodleReport, poodle_config: dict)


ast.unparse was added in 3.9

If requested, could add library astunparse-fixed to unparse 3.8

In [ ]:
import ast, json

print(ast.dump(ast.parse('a = b = 1'), indent=4))

print(ast.dump(ast.parse('a = 1'), indent=4))

In [ ]:
from copy import deepcopy
m = ast.parse('1<x<=3')

m2 = deepcopy(m)
m2.body[0].value.ops[0] = ast.Gt()

print(ast.unparse(m))
print(ast.dump(m, indent=4))
print(ast.unparse(m2))
print(ast.dump(m2, indent=4))